# 11 Sklearn Features I Sincerely Hope You Learn After Using It For a Year
## TODO
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@psco?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pineapple Supply Co.</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-three-pineapples-surrounded-by-balloons-1071882/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>